# SQL code 

[COURES problem CS50](https://cs50.harvard.edu/x/2024/psets/7/fiftyville/)

In [1]:
# Load the extension:
%load_ext sql

# Connect to DB
%sql sqlite:///fiftyville.db

Connecting to 'sqlite:///fiftyville.db'

In [2]:
%%sql
SELECT sql FROM sqlite_master WHERE type='table';

Running query in 'sqlite:///fiftyville.db'

sql
"CREATE TABLE crime_scene_reports ( id INTEGER, year INTEGER, month INTEGER, day INTEGER, street TEXT, description TEXT, PRIMARY KEY(id))"
"CREATE TABLE interviews ( id INTEGER, name TEXT, year INTEGER, month INTEGER, day INTEGER, transcript TEXT, PRIMARY KEY(id))"
"CREATE TABLE atm_transactions ( id INTEGER, account_number INTEGER, year INTEGER, month INTEGER, day INTEGER, atm_location TEXT, transaction_type TEXT, amount INTEGER, PRIMARY KEY(id))"
"CREATE TABLE bank_accounts ( account_number INTEGER, person_id INTEGER, creation_year INTEGER, FOREIGN KEY(person_id) REFERENCES people(id))"
"CREATE TABLE airports ( id INTEGER, abbreviation TEXT, full_name TEXT, city TEXT, PRIMARY KEY(id))"
"CREATE TABLE flights ( id INTEGER, origin_airport_id INTEGER, destination_airport_id INTEGER, year INTEGER, month INTEGER, day INTEGER, hour INTEGER, minute INTEGER, PRIMARY KEY(id), FOREIGN KEY(origin_airport_id) REFERENCES airports(id), FOREIGN KEY(destination_airport_id) REFERENCES airports(id))"
"CREATE TABLE passengers ( flight_id INTEGER, passport_number INTEGER, seat TEXT, FOREIGN KEY(flight_id) REFERENCES flights(id))"
"CREATE TABLE phone_calls ( id INTEGER, caller TEXT, receiver TEXT, year INTEGER, month INTEGER, day INTEGER, duration INTEGER, PRIMARY KEY(id))"
"CREATE TABLE people ( id INTEGER, name TEXT, phone_number TEXT, passport_number INTEGER, license_plate TEXT, PRIMARY KEY(id))"
"CREATE TABLE bakery_security_logs ( id INTEGER, year INTEGER, month INTEGER, day INTEGER, hour INTEGER, minute INTEGER, activity TEXT, license_plate TEXT, PRIMARY KEY(id))"


In [3]:
%%sql

-- Find description of this crime
SELECT description FROM crime_scene_reports
WHERE month = 7 AND day = 28 
AND street = 'Humphrey Street';


Running query in 'sqlite:///fiftyville.db'

description
Theft of the CS50 duck took place at 10:15am at the Humphrey Street bakery. Interviews were conducted today with three witnesses who were present at the time – each of their interview transcripts mentions the bakery.
Littering took place at 16:36. No known witnesses.


-- Two incidents happened that day. Only one is related to a thief and we have three witnesses.
-- Find interviews transcripts of witnesses



In [4]:
%%sql

-- Find interviewes transcripts of witnesses
SELECT name, transcript FROM interviews
WHERE month = 7 AND day = 28 AND year = 2023;

Running query in 'sqlite:///fiftyville.db'

name,transcript
Jose,"“Ah,” said he, “I forgot that I had not seen you for some weeks. It is a little souvenir from the King of Bohemia in return for my assistance in the case of the Irene Adler papers.”"
Eugene,"“I suppose,” said Holmes, “that when Mr. Windibank came back from France he was very annoyed at your having gone to the ball.”"
Barbara,"“You had my note?” he asked with a deep harsh voice and a strongly marked German accent. “I told you that I would call.” He looked from one to the other of us, as if uncertain which to address."
Ruth,"Sometime within ten minutes of the theft, I saw the thief get into a car in the bakery parking lot and drive away. If you have security footage from the bakery parking lot, you might want to look for cars that left the parking lot in that time frame."
Eugene,"I don't know the thief's name, but it was someone I recognized. Earlier this morning, before I arrived at Emma's bakery, I was walking by the ATM on Leggett Street and saw the thief there withdrawing some money."
Raymond,"As the thief was leaving the bakery, they called someone who talked to them for less than a minute. In the call, I heard the thief say that they were planning to take the earliest flight out of Fiftyville tomorrow. The thief then asked the person on the other end of the phone to purchase the flight ticket."
Lily,"Our neighboring courthouse has a very annoying rooster that crows loudly at 6am every day. My sons Robert and Patrick took the rooster to a city far, far away, so it may never bother us again. My sons have successfully arrived in Paris."


-- Select corresponding witnesses 

In [5]:
%%sql

-- Selecting relevant witnesses
SELECT name, transcript FROM interviews
WHERE month = 7 AND day = 28 AND year = 2023 

-- Get witnesses transcript
AND name IN ('Ruth', 'Eugene', 'Raymond') 
AND transcript LIKE '%bakery%';

Running query in 'sqlite:///fiftyville.db'

name,transcript
Ruth,"Sometime within ten minutes of the theft, I saw the thief get into a car in the bakery parking lot and drive away. If you have security footage from the bakery parking lot, you might want to look for cars that left the parking lot in that time frame."
Eugene,"I don't know the thief's name, but it was someone I recognized. Earlier this morning, before I arrived at Emma's bakery, I was walking by the ATM on Leggett Street and saw the thief there withdrawing some money."
Raymond,"As the thief was leaving the bakery, they called someone who talked to them for less than a minute. In the call, I heard the thief say that they were planning to take the earliest flight out of Fiftyville tomorrow. The thief then asked the person on the other end of the phone to purchase the flight ticket."


-- Witnesses are- Eugene, Raymond, and Ruth.

-- Ruth gave clues- The thief drove away in a car from the bakery, within 10 minutes from the theft. So, checking the license plates of cars within that timeframe. 

-- Then, checking out the names of those cars' owners. They could be suspect.

In [6]:
%%sql

-- GET bakery security logs at 10:15am 
SELECT * FROM bakery_security_logs
WHERE
activity = 'exit'
AND month = 7 
AND day = 28 
AND year = 2023

-- Select expected timeframe (between 10:15am and 10:25am)
AND hour = 10 AND minute BETWEEN 15 AND 25;

Running query in 'sqlite:///fiftyville.db'

id,year,month,day,hour,minute,activity,license_plate
260,2023,7,28,10,16,exit,5P2BI95
261,2023,7,28,10,18,exit,94KL13X
262,2023,7,28,10,18,exit,6P58WS2
263,2023,7,28,10,19,exit,4328GD8
264,2023,7,28,10,20,exit,G412CB7
265,2023,7,28,10,21,exit,L93JTIZ
266,2023,7,28,10,23,exit,322W7JE
267,2023,7,28,10,23,exit,0NTHK55


-- We have the license_plate now I should check who is the owner. 

In [7]:
%%sql

-- Select name of possible thief using people table
SELECT people.id, name, phone_number, passport_number, bakery.hour, bakery.minute
FROM people
JOIN bakery_security_logs AS bakery ON people.license_plate = bakery.license_plate
WHERE activity = 'exit'
AND month = 7 
AND day = 28 
AND year = 2023

-- Select expected timeframe (between 10:15am and 10:25am)
AND hour = 10
AND minute BETWEEN 15 AND 25

ORDER BY bakery.minute;

Running query in 'sqlite:///fiftyville.db'

id,name,phone_number,passport_number,hour,minute
221103,Vanessa,(725) 555-4692,2963008352,10,16
686048,Bruce,(367) 555-5533,5773159633,10,18
243696,Barry,(301) 555-4174,7526138472,10,18
467400,Luca,(389) 555-5198,8496433585,10,19
398010,Sofia,(130) 555-0289,1695452385,10,20
396669,Iman,(829) 555-5269,7049073643,10,21
514354,Diana,(770) 555-1861,3592750733,10,23
560886,Kelsey,(499) 555-9472,8294398571,10,23


-- Create a temporary table for future analyze named possible_suspects

In [8]:
%%sql

-- Create a temporary table possible_suspects
CREATE TEMPORARY TABLE possible_suspects AS

-- Select name of possible thief using people table
SELECT people.id, name, phone_number, passport_number, bakery.hour, bakery.minute
FROM people
JOIN bakery_security_logs AS bakery ON people.license_plate = bakery.license_plate
WHERE activity = 'exit'
AND month = 7 
AND day = 28 
AND year = 2023

-- Select expected timeframe (between 10:15am and 10:25am)
AND hour = 10
AND minute BETWEEN 15 AND 25

ORDER BY bakery.minute;

Running query in 'sqlite:///fiftyville.db'

++
||
++
++

>**Note:** <br> Temporary tables are session-specific and are dropped automatically at the end of the session.

In [9]:
%%sql 

-- Display table
SELECT * FROM possible_suspects;

Running query in 'sqlite:///fiftyville.db'

id,name,phone_number,passport_number,hour,minute
221103,Vanessa,(725) 555-4692,2963008352,10,16
686048,Bruce,(367) 555-5533,5773159633,10,18
243696,Barry,(301) 555-4174,7526138472,10,18
467400,Luca,(389) 555-5198,8496433585,10,19
398010,Sofia,(130) 555-0289,1695452385,10,20
396669,Iman,(829) 555-5269,7049073643,10,21
514354,Diana,(770) 555-1861,3592750733,10,23
560886,Kelsey,(499) 555-9472,8294398571,10,23


-- Eugene gave clues the thief was withdrawing money from the ATM on Leggett Street. 

In [10]:
%%sql

-- Find ATM withdraw the day of the stolen duck
SELECT * FROM atm_transactions
WHERE atm_location = 'Leggett Street' AND transaction_type = 'withdraw'
AND month = 7 AND day = 28 AND year = 2023;

Running query in 'sqlite:///fiftyville.db'

id,account_number,year,month,day,atm_location,transaction_type,amount
246,28500762,2023,7,28,Leggett Street,withdraw,48
264,28296815,2023,7,28,Leggett Street,withdraw,20
266,76054385,2023,7,28,Leggett Street,withdraw,60
267,49610011,2023,7,28,Leggett Street,withdraw,50
269,16153065,2023,7,28,Leggett Street,withdraw,80
288,25506511,2023,7,28,Leggett Street,withdraw,20
313,81061156,2023,7,28,Leggett Street,withdraw,30
336,26013199,2023,7,28,Leggett Street,withdraw,35


-- Create a temporary table for future analyze named suspects

In [11]:
%%sql 

-- Create a temporary table suspects
CREATE TEMPORARY TABLE suspects AS

-- Find the related name of suspects
SELECT * FROM possible_suspects
WHERE id IN (
    
    -- Select people ids 
    SELECT person_id FROM bank_accounts
    JOIN atm_transactions ON bank_accounts.account_number = atm_transactions.account_number
    
    -- Find ATM withdraw the day of the stolen duck
    WHERE atm_location = 'Leggett Street' AND transaction_type = 'withdraw'
    AND month = 7 AND day = 28 AND year = 2023
);

Running query in 'sqlite:///fiftyville.db'

++
||
++
++

In [12]:
%%sql

-- Display suspects table
SELECT * FROM suspects;

Running query in 'sqlite:///fiftyville.db'

id,name,phone_number,passport_number,hour,minute
686048,Bruce,(367) 555-5533,5773159633,10,18
467400,Luca,(389) 555-5198,8496433585,10,19
396669,Iman,(829) 555-5269,7049073643,10,21
514354,Diana,(770) 555-1861,3592750733,10,23


-- Raymond gave clues

-- As leaving the bakery, they called a person and talked for less than a minute. 

-- They asked the person on the other end to purchase the flight ticket.

-- Check whom the thief called using the specified time frame and filter for calls with a duration of less than 1 minute.

-- Create a temporary table for future analyze named thief_suspects

-- Create a temporary table for future analyze named thief_suspects

In [13]:
%%sql

-- Create a temporary table of thief_suspects
CREATE TEMPORARY TABLE thief_suspects AS

-- Get the name of people who called
SELECT * FROM suspects
WHERE phone_number IN
(  
    -- The thief called someone for less than 1 min 
    SELECT caller FROM phone_calls
    WHERE month = 7 AND day = 28 AND year = 2023 AND duration <= 60
);

Running query in 'sqlite:///fiftyville.db'

++
||
++
++

In [14]:
%%sql

-- Display table
SELECT * FROM thief_suspects

Running query in 'sqlite:///fiftyville.db'

id,name,phone_number,passport_number,hour,minute
686048,Bruce,(367) 555-5533,5773159633,10,18
514354,Diana,(770) 555-1861,3592750733,10,23


-- The thief is one of this two 
-- Next let check who took the airplane on the 29 (the day after)

In [15]:
%%sql

SELECT flights.hour, flights.minute, name, phone_number, thief_suspects.passport_number FROM passengers
JOIN thief_suspects ON passengers.passport_number = thief_suspects.passport_number
JOIN flights ON passengers.flight_id = flights.id
-- Select the corresponding day
WHERE year = 2023 
AND month = 7
AND day = 29
-- Order by departer time
ORDER BY flights.hour, flights.minute;

Running query in 'sqlite:///fiftyville.db'

hour,minute,name,phone_number,passport_number
8,20,Bruce,(367) 555-5533,5773159633
16,0,Diana,(770) 555-1861,3592750733


-- When the thief called he say that they were planning to take the earliest flight

-- Create a temporary table for future analyze named thief

In [16]:
%%sql

-- Create a temporary table named thief
CREATE TEMPORARY TABLE thief AS

SELECT flight_id, origin_airport_id, destination_airport_id, 
flights.hour, flights.minute, 
name, phone_number, thief_suspects.passport_number 
FROM passengers
JOIN thief_suspects ON passengers.passport_number = thief_suspects.passport_number
JOIN flights ON passengers.flight_id = flights.id
-- Select the corresponding day
WHERE year = 2023 
AND month = 7
AND day = 29
-- Order by departer time
ORDER BY flights.hour, flights.minute
-- SELCT the earliest flight
LIMIT 1;

Running query in 'sqlite:///fiftyville.db'

++
||
++
++

In [17]:
%%sql

-- Show thief table
SELECT * FROM thief;

Running query in 'sqlite:///fiftyville.db'

flight_id,origin_airport_id,destination_airport_id,hour,minute,name,phone_number,passport_number
36,8,4,8,20,Bruce,(367) 555-5533,5773159633


In [18]:
%%sql
-- Show thief name
SELECT name FROM thief;

Running query in 'sqlite:///fiftyville.db'

name
Bruce


-- Get the city name where the thief went

In [19]:
%%sql
-- Get the name of the name of the city the thief went
SELECT city FROM airports
JOIN thief ON thief.destination_airport_id = airports.id;

Running query in 'sqlite:///fiftyville.db'

city
New York City


-- Who is the accomplice ? Let's find who the thief asked the person on the other end of the phone to purchase the flight ticket.


In [20]:
%%sql

-- Get the name of the accomplice
SELECT name FROM people
WHERE phone_number = (
    
    -- Get the phone number of the person the thief asked to purchase the flight ticket.
    SELECT receiver FROM phone_calls
    JOIN thief ON thief.phone_number = phone_calls.caller
    WHERE year = 2023
    AND month = 7
    AND day = 28
    AND duration <= 60
);




Running query in 'sqlite:///fiftyville.db'

name
Robin
